In [ ]:
#for Google Colab
import sys
from google.colab import drive
from IPython.display import Javascript
from pprint import pprint

drive.mount('/content/drive')

###---- ROOT_PATHを記載 ----###
ROOT_PATH = '/content/drive/MyDrive/Scraping/'
sys.path.append(ROOT_PATH)

Mounted at /content/drive


In [ ]:
!pip install -q trafilatura
!pip install -q beautifulsoup4
!pip install -q requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 837.8/837.8 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.8/263.8 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 14.4 MB/s eta 0:00:00


In [ ]:
import requests
from bs4 import BeautifulSoup

# スクレイピングする基本URL
base_url = 'https://www.nomura.co.jp'

# スクレイピングするURLのリスト
urls_to_scrape = [
    'https://www.nomura.co.jp/terms/a_index.html',
    'https://www.nomura.co.jp/terms/ka_index.html',
    'https://www.nomura.co.jp/terms/sa_index.html',
    'https://www.nomura.co.jp/terms/ta_index.html',
    'https://www.nomura.co.jp/terms/na_index.html',
    'https://www.nomura.co.jp/terms/ha_index.html',
    'https://www.nomura.co.jp/terms/ma_index.html',
    'https://www.nomura.co.jp/terms/ya_index.html',
    'https://www.nomura.co.jp/terms/ra_index.html',
    'https://www.nomura.co.jp/terms/wa_index.html',
    'https://www.nomura.co.jp/terms/eng_index.html'
]

# 最終的なURLリストを保持するためのセット
final_urls = set()

# 各URLをループ処理
for url in urls_to_scrape:
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        # '/terms/japan'で始まるすべての<a>タグを見つける
        for link in soup.find_all('a', href=True):
            href = link.get('href')
            if href.startswith('/terms/japan') or href.startswith('/terms/english'):
                full_url = base_url + href
                final_urls.add(full_url)
                print(full_url)  # 確認用にURLを表示
    else:
        print(f"Failed to retrieve content from {url}")

# 重複を削除したリストに変換
unique_urls = list(final_urls)

# ユニークなURLの数と内容を表示
print(f"Number of unique URLs: {len(unique_urls)}")
print(unique_urls)


https://www.nomura.co.jp/terms/japan/a/ism_seizou.html
https://www.nomura.co.jp/terms/japan/a/A02513.html
https://www.nomura.co.jp/terms/japan/a/aitaibaibai.html
https://www.nomura.co.jp/terms/japan/a/A02965.html
https://www.nomura.co.jp/terms/japan/a/A02085.html
https://www.nomura.co.jp/terms/japan/a/A02715.html
https://www.nomura.co.jp/terms/japan/a/outperformance.html
https://www.nomura.co.jp/terms/japan/a/A02848.html
https://www.nomura.co.jp/terms/japan/a/outlook.html
https://www.nomura.co.jp/terms/japan/a/outofthemoney.html
https://www.nomura.co.jp/terms/japan/a/A01894.html
https://www.nomura.co.jp/terms/japan/a/aotenjo.html
https://www.nomura.co.jp/terms/japan/a/A01934.html
https://www.nomura.co.jp/terms/japan/a/akasanpei.html
https://www.nomura.co.jp/terms/japan/a/akaji-kokusai.html
https://www.nomura.co.jp/terms/japan/a/A02622.html
https://www.nomura.co.jp/terms/japan/a/accumulation.html
https://www.nomura.co.jp/terms/japan/a/A02621.html
https://www.nomura.co.jp/terms/japan/a/A

In [ ]:
urls = unique_urls

In [ ]:
len(urls)

3149

In [ ]:
import requests
import csv, json, os, time
from bs4 import BeautifulSoup
from trafilatura import fetch_url, extract


output_dir = ROOT_PATH + "jsonl/"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)


def fetch_all_pages_to_jsonl(jsonl_filename):
    filename = os.path.join(output_dir, jsonl_filename)
    # 抽出したページ数のカウント
    count = 0
    # 全ページ数のカウント
    total_count = 0

    with open(filename, 'w', encoding='utf_8_sig') as file:

      for url in urls:
        total_count += 1
        page_url = url
        downloaded = fetch_url(page_url)
        soup = BeautifulSoup(downloaded, 'html.parser')
        title = soup.title.text if soup.title else ''
        extracted_text = extract(downloaded, include_tables=False, target_lang='ja')

        # "。"を含まないページはフィルタリング（単語のみや英語のページが多いため）
        if not extracted_text or "。" not in extracted_text:
            # print(extracted_text)
            continue

        # jsonl形式で書き込み
        page_data = {'title': title, 'url': page_url, 'text': extracted_text}
        json.dump(page_data, file, ensure_ascii=False)
        file.write('\n')

        count += 1
        print(f"Processed {count} pages.") if count % 1000 == 0 else None
        # if total_count==10:
        #   break
        time.sleep(2)

    # 抽出したページ数を表示
    print(f"Total pages retrieved: {count}/ {total_count}")

fetch_all_pages_to_jsonl('nomura.jsonl')


Processed 1000 pages.
Processed 2000 pages.
Processed 3000 pages.
Total pages retrieved: 3149/ 3149
